# Connecting mocks to spectra.

This notebook presents QA checks of the code which assigns spectra to the mock catalogs.

## Initialize

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os
import numpy as np
import warnings
import yaml
from glob import glob

In [3]:
# Note that you need the standard desi/conda dependencies + matplotlib, 
# basemap, and healpy to run this notebook.
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import fitsio
from astropy.table import Table, Column, vstack

In [4]:
from desispec.brick import brickname as get_brickname_from_radec
from desitarget.targetmask import desi_mask, bgs_mask, mws_mask, contam_mask
from desiutil.plots import init_sky, plot_sky_binned
from desiutil.log import get_logger
log = get_logger()

ImportError: cannot import name 'contam_mask'

## Section 1: Assess the input mock catalogs.

### 1a) Read the mocks and determine the space density at the brick level.

In [ ]:
def _GRFpath(sky=False):
    if sky:
        return os.path.join(os.getenv('DESI_ROOT'), 'mocks', 'GaussianRandomField', 'v0.0.1', '2048')
    else:
        return os.path.join(os.getenv('DESI_ROOT'), 'mocks', 'GaussianRandomField', 'v0.0.5')

In [ ]:
def _BGSpath():
    return os.path.join(os.getenv('DESI_ROOT'), 'mocks', 'bgs', 'MXXL', 'desi_footprint', 'v0.0.3', 'BGS.hdf5')

In [ ]:
def _lyapath():
    return os.path.join(os.getenv('DESI_ROOT'), 'mocks', 'lya_forest', 'v0.0.2')

In [ ]:
def _mwspath(WD=False, nearby=False, faint=False):
    rootpath = os.path.join(os.getenv('DESI_ROOT'), 'mocks', 'mws')
    if WD:
        return os.path.join(rootpath, 'wd', 'v0.0.2', 'mock_wd.fits')
    if nearby:
        return os.path.join(rootpath, '100pc', 'v0.0.3', 'mock_100pc.fits')
    if faint:
        return os.path.join(rootpath, 'galaxia', 'alpha', '0.0.2_superfaint', 'bricks')
    # Default
    return os.path.join(rootpath, 'galaxia', 'alpha', 'v0.0.4', 'bricks')

In [ ]:
def read_GRF(fitsfile):
    """Read the Gaussian Random Field mocks."""
    import fitsio
    from astropy.table import Table, Column
    log.info('Reading {}'.format(fitsfile))
    dat = Table(fitsio.read(fitsfile, ext=1, upper=True))
    if 'random' not in fitsfile:
        dat.add_column(Column(name='Z', data=(dat['Z_COSMO'] + dat['DZ_RSD']).astype('f4')))
    
    wrap = np.where(dat['RA'] == 360.0)[0]
    if len(wrap) > 0:
        dat['RA'][wrap] = 0.0

    brickname = get_brickname_from_radec(dat['RA'], dat['DEC'], bricksize=0.25)
    dat.add_column(Column(name='BRICKNAME', data=brickname))
    return dat

In [ ]:
def _read_lya(lyafile):
    """Read the metadata from a single Lya file."""
    log.info('Reading {}'.format(lyafile))
    ra, dec, z = [], [], []
    ff = fitsio.FITS(lyafile)
    ff = ff[1:len(ff)]
    for h in ff:
        head = h.read_header()
        z.append(head['ZQSO'])
        ra.append(head['RA'])
        dec.append(head['DEC'])

    ra = np.array(ra)
    dec = np.array(dec)
    z = np.array(z).astype('f4')
    ra = ra * 180.0 / np.pi
    ra = ra % 360.0 #enforce 0 < ra < 360
    dec = dec * 180.0 / np.pi

    wrap = np.where(ra == 360.0)[0]
    if len(wrap) > 0:
        ra[wrap] = 0.0
    
    dat = Table()
    dat.add_column(Column(name='RA', data=ra))
    dat.add_column(Column(name='DEC', data=dec))
    dat.add_column(Column(name='Z', data=z))
    
    brickname = get_brickname_from_radec(dat['RA'], dat['DEC'], bricksize=0.25)
    dat.add_column(Column(name='BRICKNAME', data=brickname))

    return dat

In [ ]:
def read_lya(nread=None):
    """Read the Lyman-alpha mocks."""
    import multiprocessing
    nproc = max(1, multiprocessing.cpu_count() // 2)

    lyafiles = glob(os.path.join(_lyapath(), 'simpleSpec_*.fits.gz'))
    if nread:
        lyafiles = lyafiles[:nread]
    log.info('Reading metadata for {} Lya files'.format(len(lyafiles)))

    p = multiprocessing.Pool(nproc)
    dat = p.map(_read_lya, lyafiles)
    p.close()
    
    return vstack(dat)

In [ ]:
def read_BGS(hdf5file):
    """Read the MXXL/BGS mock."""
    import h5py
    log.info('Reading {}'.format(hdf5file))
    f = h5py.File(hdf5file)
    ra  = f['Data/ra'][...].astype('f8') % 360.0
    dec = f['Data/dec'][...].astype('f8')
    zobs = f['Data/z_obs'][...].astype('f4')
    rmag = f['Data/app_mag'][...].astype('f8')
    log.warning('Note: the BGS mock has been trimmed to r_{SDSS}<20.1, which is not quite right.')
    these = rmag < 20.1
    
    dat = Table()
    dat.add_column(Column(name='RA', data=ra[these]))
    dat.add_column(Column(name='DEC', data=dec[these]))
    dat.add_column(Column(name='Z', data=zobs[these]))

    brickname = get_brickname_from_radec(dat['RA'], dat['DEC'], bricksize=0.25)
    dat.add_column(Column(name='BRICKNAME', data=brickname))

    return dat

In [ ]:
def mock_density(allcat, objtype, bricksize=0.25):
    """Determine the target density of the input mocks (used for QA)."""
    from desispec.brick import Bricks
    
    brickarea = bricksize**2 # deg2

    B = Bricks(bricksize=0.25)
    brickname = np.concatenate(B._brickname)
    ubrickname = np.unique(brickname)
    nbrick = len(ubrickname)
    
    density = Table()
    density.add_column(Column(name='BRICKNAME', length=nbrick, dtype='U10'))
    density.add_column(Column(name='DENS_BGS', length=nbrick, dtype='f4'))
    density.add_column(Column(name='DENS_ELG', length=nbrick, dtype='f4'))
    density.add_column(Column(name='DENS_LRG', length=nbrick, dtype='f4'))
    density.add_column(Column(name='DENS_QSO', length=nbrick, dtype='f4'))
    density.add_column(Column(name='DENS_LYA', length=nbrick, dtype='f4'))
    density.add_column(Column(name='DENS_SKY', length=nbrick, dtype='f4'))

    density['BRICKNAME'] = ubrickname
    
    log.info('Gathering brick info for {} mock catalog(s).'.format(len(np.atleast_1d(objtype))))
    for obj in np.atleast_1d(objtype.upper()):
        ubricks, nobj = np.unique(data[obj]['BRICKNAME'], return_counts=True)
        log.info('Estimating density for {} bricks of object type {}.'.format(
            len(ubricks), obj))
        # Is there any way to make this faster?!?
        for thisbrick, dens in zip(ubricks, nobj):
            this = np.where(thisbrick == ubrickname)[0]
            density['DENS_{}'.format(obj)][this] = dens / brickarea
            
    return density

In [ ]:
# Dictionary to hold all the mocks.
data = dict()

In [ ]:
data['BGS'] = read_BGS(_BGSpath())

In [ ]:
for mock in ('ELG', 'LRG', 'QSO'):
    mockfile = os.path.join(_GRFpath(), '{}.fits'.format(mock))
    data[mock] = read_GRF(mockfile)

In [ ]:
data['LYA'] = read_lya(nread=4)

In [ ]:
mockfile = os.path.join(_GRFpath(sky=True), 'random.fits')
data['SKY'] = read_GRF(mockfile)

In [ ]:
# Too slow!
if False:
    mockdensity = mock_density(data, ('BGS', 'ELG', 'LRG', 'QSO', 'LYA', 'SKY'))

### 1b) Generate basic QA plots.

In [ ]:
def qamock_sky(cat, objtype, ax=None):
    fig, ax = plt.subplots(1, 2, figsize=(12, 4))
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        basemap = init_sky(galactic_plane_color='k', ax=ax[0]);
        plot_sky_binned(cat['RA'], cat['DEC'], verbose=False, 
                        clip_lo='!1', cmap='jet', plot_type='healpix', 
                        label=r'{} (targets/deg$^2$)'.format(objtype), 
                        basemap=basemap)
    if 'Z' in cat.keys():
        ax[1].hist(cat['Z'], bins=100, histtype='stepfilled', alpha=0.6, label=objtype)
        ax[1].set_xlabel('Redshift')
        ax[1].set_xlim(0, 3)
        ax[1].yaxis.set_major_formatter(plt.NullFormatter())
        ax[1].legend(loc='upper right', frameon=False)
    else:
        ax[1].axis('off')
    fig.subplots_adjust(wspace=0.2)

In [ ]:
for obj in ('BGS', 'ELG', 'LRG'):
    qamock_sky(data[obj], obj)

In [ ]:
for obj in ('QSO', 'LYA'):
    qamock_sky(data[obj], obj)

In [ ]:
for obj in np.atleast_1d('SKY'):
    qamock_sky(data[obj], obj)

### 1c) Clean up to free up memory.

In [ ]:
del data

### 2a) Generate the target and truth catalogs.

In [ ]:
def write_yaml(bounds=(203, 204, 0, 1), outfile='mock_input.yaml', verbose=False):
    """Write a yaml file on-the-fly for *all* target types."""
    log.info('Writing {}'.format(outfile))
    yf = open(outfile, 'w')
    yf.write('dust_dir: {}\n'.format(os.path.join(os.getenv('DUST_DIR'), 'maps')))
    yf.write('decals_brick_info: {}\n'.format(os.path.join(os.getenv('DESI_ROOT'), 'target', 
                                                           'catalogs', 'brick-info-dr3.v0.0.1.fits')))
    yf.write('subset:\n')
    yf.write('    ra_dec_cut: True\n')
    for blabel, bbound in zip(('min_ra', 'max_ra', 'min_dec', 'max_dec'), bounds):
        yf.write('    {}: {:g}\n'.format(blabel, bbound))
    yf.write('sources:\n')
    yf.write('    ELG: {\n')
    yf.write('        target_name: ELG,\n')
    yf.write('        mock_dir_name: {},\n'.format(_GRFpath()))
    yf.write('        format: gaussianfield,\n')
    yf.write('        density: 2400,\n')
    yf.write('        contam: {\n')
    yf.write('            STAR: 240,\n')
    yf.write('        }\n')
    yf.write('    }\n')
    yf.write('    MWS_MAIN: {\n')
    yf.write('        target_name: MWS_MAIN,\n')
    yf.write('        mock_dir_name: {},\n'.format(_mwspath()))
    yf.write('        format: galaxia,\n')
    yf.write('    }\n')
    yf.write('    FAINTSTAR: {\n')
    yf.write('        target_name: FAINTSTAR,\n')
    yf.write('        mock_dir_name: {},\n'.format(_mwspath(faint=True)))
    yf.write('        format: galaxia,\n')
    yf.write('        magcut: 24.0\n')
    yf.write('    }\n')
    yf.write('    QSO: {\n')
    yf.write('        target_name: QSO,\n')
    yf.write('        mock_dir_name: {},\n'.format(_GRFpath()))
    yf.write('        format: gaussianfield,\n')
    yf.write('        density: 120,\n')
    yf.write('        contam: {\n')
    yf.write('            GALAXY: 27,\n')
    yf.write('            STAR: 63,\n')
    yf.write('        }\n')
    yf.write('    }\n')
    yf.close()    
    if verbose:
        print()
        with(open(outfile, 'r')) as yf:
            print(yf.read())

In [ ]:
def build_targets_truth(prefix='qatargets', seed=123, nproc=3, verbose=True, 
                        bounds=(203, 203.25, 0.125, 0.625), bricksize=0.25,
                        output_dir=None):
    """Simple wrapper on desitarget.mock.targets_truth.
    
    Prefix can also optionally be an object type (e.g., ELG).
    """
    from desitarget.mock.build import targets_truth
    
    if output_dir is None:
        output_dir = os.path.join(mockpath, '{}'.format(prefix))
    try:
        os.stat(output_dir)
    except:
        os.makedirs(output_dir)

    config_file = os.path.join(mockpath, '{}_input.yaml'.format(prefix))
    write_yaml(bounds=bounds, outfile=config_file, verbose=verbose)
    
    with open(config_file, 'r') as pfile:
        params = yaml.load(pfile)
    targets_truth(params, output_dir, realtargets=None, seed=seed, verbose=verbose, 
                  nproc=nproc, bricksize=bricksize, outbricksize=bricksize)

In [ ]:
# Specify root path and the (ra,dec) boundaries.
prefix = 'qatargets'
mockpath = os.path.join(os.getenv('DESI_ROOT'), 'datachallenge', prefix)
radec_bounds = (203, 203.25, 0.125, 0.375)
# radec_bounds = (203, 203.75, 0.125, 0.875)
bricksize = 0.25

In [ ]:
# Build the mock catalogs.
build_targets_truth(seed=123, bounds=radec_bounds, prefix=prefix)

### 2b) Sanity check the output files.

In [ ]:
def get_targtruth_files(prefix='qatargets'):
    output_dir = os.path.join(mockpath, '{}'.format(prefix))
    targfiles = glob(os.path.join(output_dir, '???', 'targets-*.fits'))#[:1]
    truthfiles = glob(os.path.join(output_dir, '???', 'truth-*.fits'))#[:1]
    return targfiles, truthfiles

In [ ]:
def sanity_check_files(prefix='qatargets'):
    """Simple script to do some basic sanity checks of the files on disk."""
    targfiles, truthfiles = get_targtruth_files(prefix)
    log.info('Sanity checking {} target and {} truth files in directory {}.'.format(
        len(targfiles), len(truthfiles), prefix))
             
    if len(targfiles) != len(truthfiles):
        log.warning('Mismatch in the number of target (N={}) and truth (N={}) files!').format(
            len(targfiles), len(truthfiles))

    for targfile, truthfile in zip(np.atleast_1d(targfiles), np.atleast_1d(truthfiles)):
        targ = fitsio.FITS(targfile)
        truth = fitsio.FITS(truthfile)
        ntarg = targ[1].get_nrows()
        ntruth = truth[2].get_nrows()
        wave_dims = truth[0].get_dims()
        flux_dims = truth[1].get_dims()

        if (ntarg != ntruth):
            log.warning('{}: {} objects in TRUTH table, {} objects in TARGETS table.'.format(
                os.path.basename(targfile), ntruth, ntarg))

        if (flux_dims[0] != ntarg):
            log.warning('{}: {} objects in TRUTH table, {} spectra.'.format(
                os.path.basename(targfile), ntruth, flux_dims[0]))
            
        if (flux_dims[1] != wave_dims[0]):
            log.warning('{}}: {} pixels in wavelength vector, {} pixels in flux spectra.'.format(
                os.path.basename(targfile), wave_dims[0], flux_dims[1]))

In [ ]:
sanity_check_files(prefix)

### Read the data and derive the target densities.

In [ ]:
def read_targets_truth(prefix):
    """Simple wrapper script to gather up all the target and truth catalogs."""
    targfiles, truthfiles = get_targtruth_files(prefix)
    targets, truth = [], []
    for targfile, truthfile in zip(np.atleast_1d(targfiles), np.atleast_1d(truthfiles)):
        targets.append(Table(fitsio.read(targfile, ext=1)))
        truth.append(Table(fitsio.read(truthfile, ext=2)))
    targets = vstack(targets)
    truth = vstack(truth)
    log.info('Read {} targets from {} bricks.'.format(len(targets), len(targfiles)))
    return targets, truth

In [ ]:
def target_density(prefix, targets, truth):
    """Determine the target density of each object type (including contaminants)."""
    
    brickarea = bricksize**2 # deg2
    
    allbricks = targets['BRICKNAME'].data
    ubricks = set(allbricks)
    nbrick = len(ubricks)
    
    density = Table()
    density.add_column(Column(name='BRICKNAME', length=nbrick, dtype='U10'))
    density.add_column(Column(name='RA', length=nbrick, dtype='f4'))
    density.add_column(Column(name='DEC', length=nbrick, dtype='f4'))
    # ELGs
    density.add_column(Column(name='ELG_TARGETS', length=nbrick, dtype='f4'))
    density.add_column(Column(name='ELG_IS_LOZ', length=nbrick, dtype='f4'))
    density.add_column(Column(name='ELG_IS_HIZ', length=nbrick, dtype='f4'))
    density.add_column(Column(name='ELG_IS_STAR', length=nbrick, dtype='f4'))
    # QSOs
    density.add_column(Column(name='QSO_TARGETS', length=nbrick, dtype='f4'))
    density.add_column(Column(name='QSO_IS_GALAXY', length=nbrick, dtype='f4'))
    density.add_column(Column(name='QSO_IS_STAR', length=nbrick, dtype='f4'))

    for ii, brick in enumerate(ubricks):
        indx = np.where(brick == allbricks)[0]
        density['BRICKNAME'][ii] = brick
        density['RA'][ii] = float(brick[:4]) / 10.0
        density['DEC'][ii] = float(brick[5:9]) / 10.0
        # ELGs
        mask = (targets['DESI_TARGET'][indx] & desi_mask.mask('ELG')) != 0
        density['ELG_TARGETS'][ii] = np.sum(mask) / brickarea
        density['ELG_IS_LOZ'][ii] = np.sum(mask & (truth['TRUEZ'][indx] < 0.6)) / brickarea
        density['ELG_IS_HIZ'][ii] = np.sum(mask & (truth['TRUEZ'][indx] > 1.6)) / brickarea
        density['ELG_IS_STAR'][ii] = np.sum(mask & (truth['TRUESPECTYPE'][indx] == bytes('STAR', 'utf-8'))) / brickarea
        # QSOs
        mask = (targets['DESI_TARGET'][indx] & desi_mask.mask('QSO')) != 0
        density['QSO_TARGETS'][ii] = np.sum(mask) / brickarea
        density['QSO_IS_GALAXY'][ii] = np.sum(mask & (truth['TRUESPECTYPE'][indx] == bytes('GALAXY', 'utf-8'))) / brickarea
        density['QSO_IS_STAR'][ii] = np.sum(mask & (truth['TRUESPECTYPE'][indx] == bytes('STAR', 'utf-8'))) / brickarea

    return density

In [ ]:
targets, truth = read_targets_truth(prefix)

In [ ]:
density = target_density(prefix, targets, truth)
print(density['QSO_TARGETS'])

### 2c) Make a variety of QAplots.

In [ ]:
def _grlim():
    return (-0.5, 2.0)

In [ ]:
def _rzlim():
    return (-0.5, 2.2)

In [ ]:
def _rW1lim():
    return (-2, 6)

In [ ]:
def _grzrange():
    return (16, 24)

In [ ]:
def _rmagrange():
    rmagrange = (16, 24)
    drmag = 0.1
    nrmagbins = int((rmagrange[1] - rmagrange[0]) / drmag)
    return rmagrange, nrmagbins

In [ ]:
def flux2colors(cat):
    """Convert DECam/WISE fluxes to magnitudes and colors."""
    colors = dict()
    with warnings.catch_warnings(): # ignore missing fluxes (e.g., for QSOs)
        warnings.simplefilter('ignore')
        for ii, band in zip((1, 2, 4), ('g', 'r', 'z')):
            colors[band] = 22.5 - 2.5 * np.log10(cat['DECAM_FLUX'][..., ii].data)
        for ii, band in zip((0, 1), ('W1', 'W2')):
            colors[band] = 22.5 - 2.5 * np.log10(cat['WISE_FLUX'][..., ii].data)
        colors['grz'] = 22.5-2.5*np.log10((cat['DECAM_FLUX'][..., 1] + 
                                           0.8 * cat['DECAM_FLUX'][..., 2] +
                                           0.5 * cat['DECAM_FLUX'][..., 4]).data / 2.3)
        colors['W'] = 22.5-2.5*np.log10(0.75 * cat['WISE_FLUX'][..., 0] + 
                                        0.25 * cat['WISE_FLUX'][..., 1]) # for QSOs

    colors['gr'] = colors['g'] - colors['r']
    colors['rz'] = colors['r'] - colors['z']
    colors['rW1'] = colors['r'] - colors['W1']
    colors['W1W2'] = colors['W1'] - colors['W2']
    
    return colors

In [ ]:
def elg_colorbox(ax):
    """Draw the ELG selection box."""
    rmaglim = 23.4
    grlim = ax.get_ylim()
    coeff0, coeff1 = (1.15, -0.15), (-1.2, 1.6)
    rzmin, rzpivot = 0.3, (coeff1[1] - coeff0[1]) / (coeff0[0] - coeff1[0])
    verts = [(rzmin, grlim[0]),
             (rzmin, np.polyval(coeff0, rzmin)),
             (rzpivot, np.polyval(coeff1, rzpivot)),
             ((grlim[0] - 0.1 - coeff1[1]) / coeff1[0], grlim[0] - 0.1)
            ]
    ax.add_patch(Polygon(verts, fill=False, ls='--', color='k'))
    return rmaglim

In [ ]:
def qso_colorbox(ax, plottype='grz'):
    """Draw the QSO selection boxes."""
    rmaglim = 22.7
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    if plottype == 'grz-r':
        verts = [(xlim[0]-0.05, 17.0),
                 (22.7, 17.0),
                 (22.7, ylim[1]+0.05),
                 (xlim[0]-0.05, ylim[1]+0.05)
                ]
    if plottype == 'rW1-rz':
        verts = None
        ax.axvline(x=-0.3, ls='--', color='k')
        ax.axvline(x=1.3, ls='--', color='k')

    if plottype == 'gr-rz':
        verts = [(-0.3, 1.3),
                 (1.1, 1.3),
                 (1.1, ylim[0]-0.05),
                 (-0.3, ylim[0]-0.05)
                ]
    if verts:
        ax.add_patch(Polygon(verts, fill=False, ls='--', color='k'))

In [ ]:
def qaplot_qso_colormag(targets, truth=None, contam=False):
    """Make a few different color-color plots for QSOs."""
    
    objtype = 'QSO'
    
    iobj = np.where((targets['DESI_TARGET'] & desi_mask.mask(objtype)) != 0)[0]
    if truth:
        cat = flux2colors(truth[iobj])
    else:
        cat = flux2colors(targets[iobj])

    fig, ax = plt.subplots(1, 3, figsize=(14, 4))

    if contam:
        for cc in np.atleast_1d(contam):
            these = np.where(truth['TEMPLATETYPE'].data[iobj] == bytes(cc, 'utf-8'))[0]
            if len(these) > 0:
                ax[0].scatter(cat['rz'][these], cat['gr'][these], 
                              s=10, alpha=0.7, label=cc)
            ax[0].legend(loc='upper right')
    else:
        ax[0].scatter(cat['rz'], cat['gr'], s=10, alpha=0.7)
    ax[0].set_xlabel('$r - z$')
    ax[0].set_ylabel('$g - r$')
    ax[0].set_xlim(_rzlim())
    ax[0].set_ylim(_grlim())
    qso_colorbox(ax[0], 'gr-rz')

    if contam:
        for cc in np.atleast_1d(contam):
            these = np.where(truth['TEMPLATETYPE'].data[iobj] == bytes(cc, 'utf-8'))[0]
            if len(these) > 0:
                ax[1].scatter(cat['r'][these], cat['grz'][these], 
                              s=10, alpha=0.7, label=cc)
    else:
        ax[1].scatter(cat['r'], cat['grz'], s=10, alpha=0.7)
    ax[1].set_xlabel('$r$')
    ax[1].set_ylabel('$grz$')
    ax[1].set_xlim(_rmagrange()[0])
    ax[1].set_ylim(_grzrange())
    qso_colorbox(ax[1], 'grz-r')
    
    magrange, nbins = _rmagrange()
    nn, bins, _ = ax[2].hist(cat['r'], bins=nbins, range=magrange, align='mid')
    ax[2].axvline(x=22.7, ls='--', color='red')
    ax[2].set_xlabel('r')
    ax[2].set_ylabel('Number of Objects')
    
    if truth:
        plt.suptitle('{} - All Targets, True Photometry'.format(objtype.upper()))
    else:
        plt.suptitle('{} - All Targets, Observed Photometry'.format(objtype.upper()))
        
    fig.subplots_adjust(wspace=0.3)

In [ ]:
def qaplot_elg_colormag(targets, truth=None, contam=False):
    """Make a few different color-color plots for ELGs."""
    
    objtype = 'ELG'
    
    iobj = np.where((targets['DESI_TARGET'] & desi_mask.mask(objtype)) != 0)[0]
    if truth:
        cat = flux2colors(truth[iobj])
    else:
        cat = flux2colors(targets[iobj])

    fig, ax = plt.subplots(1, 2, figsize=(8, 3))

    if contam:
        for cc in np.atleast_1d(contam):
            these = np.where(truth['TEMPLATETYPE'].data[iobj] == bytes(cc, 'utf-8'))[0]
            if len(these) > 0:
                ax[0].scatter(cat['rz'][these], cat['gr'][these], 
                              s=10, alpha=0.7, label=cc)
            ax[0].legend(loc='upper right')
    else:
        ax[0].scatter(cat['rz'], cat['gr'], s=10, alpha=0.7)
    ax[0].set_xlabel('$r - z$')
    ax[0].set_ylabel('$g - r$')
    ax[0].set_xlim(_rzlim())
    ax[0].set_ylim(_grlim())
    elg_colorbox(ax[0])

    magrange, nbins = _rmagrange()
    nn, bins, _ = ax[1].hist(cat['r'], bins=nbins, range=magrange, align='mid')
    ax[1].axvline(x=23.4, ls='--', color='red')
    ax[1].set_xlabel('r')
    ax[1].set_ylabel('Number of Objects')
    
    if truth:
        plt.suptitle('{} - All Targets, True Photometry'.format(objtype.upper()))
    else:
        plt.suptitle('{} - All Targets, Observed Photometry'.format(objtype.upper()))
        
    fig.subplots_adjust(wspace=0.3)

In [ ]:
def qaplot_elg_density(density):
    """Visualize the ELG target densities, including contaminants."""
    bins = 20
    dens_all = 2160
    dens_loz = dens_all*0.05
    dens_hiz = dens_all*0.05
    dens_star = dens_all*0.1
    contam_range = (0, dens_star*1.2)
    
    fig, ax = plt.subplots(1, 3, figsize=(10, 3))
    ax[0].hist(density['ELG_TARGETS'], bins=bins, range=(dens_all-600, dens_all+600), 
               label=['ELG - All'])
    ax[0].axvline(x=dens_all, ls='--', color='k')
    ax[0].legend(loc='upper right')

    ax[1].hist(density['ELG_IS_LOZ'], bins=bins, range=contam_range, 
               alpha=0.6, label=['ELG at z<0.6'], 
               ls='--', lw=2)
    ax[1].hist(density['ELG_IS_HIZ'], bins=bins, range=contam_range,
               alpha=0.3, label=['ELG at z>1.6'], 
               ls='-', lw=2)
    ax[1].axvline(x=dens_loz, ls='--', color='k')
    ax[1].legend(loc='upper right')  
    
    ax[2].hist(density['ELG_IS_STAR'], bins=bins, range=contam_range,
               alpha=0.8, label=['STAR'])
    ax[2].axvline(x=dens_star, ls='--', color='k')
    ax[2].legend(loc='upper right')

In [ ]:
def qaplot_qso_density(density):
    """Visualize the QSO (tracer + Lya) target densities, including contaminants."""
    bins = 20
    dens_all = 120
    dens_star = 63
    dens_galaxy = 27
    contam_range = (0, dens_star*1.2)
    
    #print(density['QSO_TARGETS'], density['QSO_IS_STAR'], density['QSO_IS_GALAXY'])
    
    fig, ax = plt.subplots(1, 3, figsize=(10, 3))
    ax[0].hist(density['QSO_TARGETS'], bins=bins, range=(dens_all-40, dens_all+40), 
               label=['QSO - All'])
    ax[0].axvline(x=dens_all, ls='--', color='k')
    ax[0].legend(loc='upper right')

    ax[1].hist(density['QSO_IS_STAR'], bins=bins, range=contam_range, 
               alpha=0.6, label=['GALAXY'], 
               ls='--', lw=2)
    ax[1].axvline(x=dens_star, ls='--', color='k')
    ax[1].legend(loc='upper right')  
    
    ax[2].hist(density['QSO_IS_GALAXY'], bins=bins, range=contam_range,
               alpha=0.8, label=['STAR'])
    ax[2].axvline(x=dens_galaxy, ls='--', color='k')
    ax[2].legend(loc='upper right')

In [ ]:
def qaplot_targets_skymap(cat, objtype):
    fig, ax = plt.subplots()
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        basemap = init_sky(galactic_plane_color='k', ax=ax);
        plot_sky_binned(cat['RA'], cat['DEC'], verbose=False, 
                        clip_lo='!1', cmap='jet', plot_type='healpix', 
                        label=r'{} (targets/deg$^2$)'.format(objtype), 
                        basemap=basemap)

In [ ]:
def qaplot_targets_sky(targets, truth, objtype='ELG', contam=None):
    iobj = np.where((targets['DESI_TARGET'] & desi_mask.mask(objtype)) != 0)[0]
    fig, ax = plt.subplots()
    if contam:
        for cc in np.atleast_1d(contam):
            these = np.where(truth['TEMPLATETYPE'].data[iobj] == bytes(cc, 'utf-8'))[0]
            if len(these) > 0:
                ax.scatter(targets['RA'][iobj][these], targets['DEC'][iobj][these], 
                              s=10, alpha=0.7, label=cc)
                ax.legend(loc='upper right')
    else:
        ax.scatter(targets['RA'][iobj], targets['DEC'][iobj], 
                      s=10, alpha=0.7)
        ax.set_title(objtype.upper())

#### ELGs.

In [ ]:
objtype = 'ELG'
qaplot_targets_sky(targets, truth, objtype, contam=('ELG', 'QSO', 'STAR'))

In [ ]:
# Would be nice if the color-color plots were color-coded by r-band magnitude.
qaplot_elg_colormag(targets)                                             # All targets, observed (noisy) photometry.
qaplot_elg_colormag(targets, truth=truth)                                # All targets, true (noiseless) photometry.
qaplot_elg_colormag(targets, truth=truth, contam=['ELG', 'STAR', 'QSO']) # Contaminants.
qaplot_elg_density(density)

#### QSOs

In [ ]:
objtype = 'QSO'
qaplot_targets_sky(targets, truth, objtype, contam=('ELG', 'QSO', 'STAR'))

In [ ]:
qaplot_qso_colormag(targets)                                             # All targets, observed (noisy) photometry.
qaplot_qso_colormag(targets, truth=truth)                                # All targets, true (noiseless) photometry.
qaplot_qso_colormag(targets, truth=truth, contam=['ELG', 'STAR', 'QSO']) # Contaminants.
qaplot_qso_density(density)